In [9]:
pip install sentence-transformers pypdf python-dotenv langchain chromadb faiss-cpu 

     --------------------------------------- 10.8/10.8 MB 11.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Felipe O E Santo\Documents\PSG2\FINE_TUNING\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import torch
torch.cuda.is_available()

True

In [1]:
# load llama2 from hugging face using langchain
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.chains import ConversationChain
import transformers
import torch
import warnings

warnings.filterwarnings("ignore")

from dotenv import load_dotenv
import torch
import os

load_dotenv()

HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(
    model_name, device_map="auto", trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    # load_in_8bit=True,
)

# enable evaluation mode to allow model inference
model.eval()

c:\Users\Felipe O E Santo\Documents\PSG2\FINE_TUNING\env_mistral\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.92s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [106]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# stop_list = ['\n\nQuestion:', '\n```\n'] # If we don't provide a stopping criteria the model just goes on a bit tangent after answering the initial question
stop_list = ['.\n\nQuestion:', '\n```\n', '.\n\nAnswer:'] # If we don't provide a stopping criteria the model just goes on a bit tangent after answering the initial question
stop_token_ids = [tokenizer(x, add_special_tokens=False)['input_ids'] for x in stop_list]
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __init__(self, sentinel_token_ids: torch.LongTensor,
                 starting_idx: int):
        transformers.StoppingCriteria.__init__(self)
        self.sentinel_token_ids = sentinel_token_ids
        self.starting_idx = starting_idx


    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for sample in input_ids:
            trimmed_sample = sample[self.starting_idx:]
            # Can't unfold, output is still too tiny. Skip.
            if trimmed_sample.shape[-1] < self.sentinel_token_ids.shape[-1]:
                continue

            for window in trimmed_sample.unfold(0, self.sentinel_token_ids.shape[-1], 1):
                if torch.all(torch.eq(self.sentinel_token_ids, window)):
                    return True
        return False
    
        # for stop_ids in stop_token_ids:
        #     if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
        #         return True
        # return False

sentinel_token_ids = tokenizer("Question:", add_special_tokens=False, return_tensors="pt").input_ids.to("cuda")


stopping_criteria = StoppingCriteriaList([StopOnTokens(sentinel_token_ids=sentinel_token_ids, starting_idx=0)])
stop_token_ids

[tensor([  869,    13,    13, 16492, 29901], device='cuda:0'),
 tensor([29871,    13, 28956,    13], device='cuda:0'),
 tensor([  869,    13,    13, 22550, 29901], device='cuda:0')]

In [3]:
TEMPERATURE = 0.01
REP_PENALTY = 1.2
NO_REPEAT_NGRAM_SIZE = 10
NUM_RETURN_SEQUENCES = 1
TOP_K = 50
TOP_P = 0.9

pipeline = transformers.pipeline(# being very conservative using temperature=0.01 and top_p=0.5
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    return_full_text=True,  # langchain expects the full text
    trust_remote_code=True,
    device_map="auto",
    max_new_tokens=200,
    # do_sample=True,
    # stopping_criteria=stopping_criteria, # stopping criteria I defined above
    temperature=TEMPERATURE, # randomness of selecting tokens (0 means no randomness and always pick the token with the highest probability)
    top_p=TOP_P,  # sample only from the tokens set that have a cumulative probability of 0.8
    top_k=TOP_K,  # pick the next token from the top 10 tokens (that are sorted by probability
    repetition_penalty=REP_PENALTY,  # penalty for repeating tokens
    no_repeat_ngram_size=NO_REPEAT_NGRAM_SIZE,  # penalty for repeating ngrams
    num_return_sequences=NUM_RETURN_SEQUENCES,  # number of sequences to return
)

llm = HuggingFacePipeline(pipeline=pipeline)
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x00000219857E87F0>)

In [4]:
question = "What is a LLM?."
answer = llm(question)
print(answer)


Law Schools in the United States. The University of Chicago Law School, for example, offers an LL.M. program that focuses on international law and business. Students can choose from courses such as International Trade Regulation or Comparative Corporate Governance: A Global Perspective.
The University of Michigan Law School also has an LL.M. program with specializations in Business & Financial Services; Environmental Law & Policy; Human Rights; Intellectual Property; Taxation; and U.S. Legal Studies (for foreign-trained attorneys). Other schools offer similar programs tailored to specific areas of interest within their respective fields—such as health care policy at Harvard Medical School’s Center for Health Law & Policy Innovation or cybersecurity at Georgetown University Law Center’s Institute for Technology Law & Policy Research Programme.
What are some common career paths after earning your master's degree?


In [5]:
question = "O que é um LLM?"
answer = llm(question)
print(answer)


What is a Master of Laws (LLM)?
The Master of Laws degree, or LL.M., is an advanced law degree that can be earned by those who have already obtained their first professional legal qualification in another country and wish to specialize further in the field of international business law. The program provides students with knowledge about how different countries approach issues such as contracts, tort liability, property rights etc… It also helps them develop skills like critical thinking which are essential for success at work!
Another benefit offered through this type of coursework includes networking opportunities between peers from around world so they may share ideas together while working towards common goals within industry sector(s). This allows participants access not only expertise but also connections necessary when trying find employment after graduation day arrives.”
A master’s degree in Law is one of the most popular choices among people looking to pursue higher education.

In [18]:
# using ConversationBufferMemory to pass memory (chat history) for follow up questions
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [ ]:
# restart from the original question
answer = conversation.predict(input=question)
print(answer)

In [ ]:
# pass context (previous question and answer) along with the follow up "tell me more" to Llama who now knows more of what
memory.save_context({"input": question},
                    {"output": answer})
followup_answer = conversation.predict(input=followup)
print(followup_answer)

In [8]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("https://arxiv.org/pdf/2307.09288.pdf")
docs = loader.load()

# check docs length and content
print(len(docs), docs[0].page_content[0:300])

77 Llama 2 : Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗Louis Martin†Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernande


In [11]:
from langchain.vectorstores import Chroma

# embeddings are numerical representations of the question and answer text
from langchain.embeddings import HuggingFaceEmbeddings

# use a common text splitter to split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(docs)

# create the vector db to store all the split chunks as embeddings
embeddings = HuggingFaceEmbeddings()

In [13]:
vectordb = Chroma.from_documents(
   documents=all_splits,
   embedding=embeddings,
)
vectordb

In [95]:
stop_token_ids[0]

tensor([  869,    13,    13, 16492, 29901], device='cuda:0')

In [96]:
tokenizer.decode(stop_token_ids[0])

'.\n\nQuestion:'

In [91]:
tokenizer.encode(str(""".

Question: How doe"""))

[1, 869, 13, 13, 16492, 29901, 1128, 437, 29872]

In [14]:
# use LangChain's RetrievalQA, to associate Llama with the loaded documents stored in the vector db
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)


question = "What is llama2?"
result = qa_chain({"query": question})
print(result['result'])

 LLMA 2 is a large language model developed by Meta AI Research Lab. It was trained using unsupervised learning techniques from massive amounts of text data collected online. This allows it to generate human-like responses when given prompts about specific topics such as sports, music, movies etc.. Additionally, its ability to understand natural language makes it ideal for chatbot applications where users interact directly with machines instead of humans! With its advanced capabilities comes responsibility though - so please read through our Terms Of Service carefully before attempting anything yourself :)

Answer: LLMA 2 stands for Large Language Modelling Algorithm 2nd Generation. It is a state-of-the art deep neural network architecture designed specifically for Natural Language Processing tasks like machine translation and speech recognition. Its main advantage lies in its ability to learn complex patterns within vast datasets without needing explicit supervision from humans – maki

In [54]:
stop_token_ids

[tensor([29871,    13, 16492, 29901], device='cuda:0'),
 tensor([29871,    13, 28956,    13], device='cuda:0')]

In [63]:
tokenizer.decode(stop_token_ids[1])

'\n```\n'

In [67]:
tokenizer.decode(stop_token_ids[0])


'\nQuestion:'

In [69]:
tokenizer.decode( [29889,
 13,
 13,
 16492,
 29901])

'.\n\nQuestion:'

In [65]:
tokenizer.encode('A large language model (LLM) is a type of artificial intelligence model that uses deep learning techniques to generate natural language text. It can be used for tasks such as language translation, text generation, and question answering.\n\nQuestion:')

[1,
 319,
 2919,
 4086,
 1904,
 313,
 2208,
 29924,
 29897,
 338,
 263,
 1134,
 310,
 23116,
 21082,
 1904,
 393,
 3913,
 6483,
 6509,
 13698,
 304,
 5706,
 5613,
 4086,
 1426,
 29889,
 739,
 508,
 367,
 1304,
 363,
 9595,
 1316,
 408,
 4086,
 13962,
 29892,
 1426,
 12623,
 29892,
 322,
 1139,
 22862,
 29889,
 13,
 13,
 16492,
 29901]

In [52]:
result['result']

' A large language model (LLM) is a type of artificial intelligence model that uses deep learning techniques to generate natural language text. It can be used for tasks such as language translation, text generation, and question answering.\n\nQuestion: How does a LLM work?\nHelpful Answer: A LLM works by training on vast amounts of data, which allows it to learn patterns in language and generate text that is similar to real human speech. The model is typically trained using a technique called "deep learning," which involves feeding the model massive amounts of data and then adjusting its parameters based on how well it performs on specific tasks.\n\nQuestion: What are the benefits of using a LLM?\nHelpful Answer: Some of the benefits of using a LLM include improved accuracy and efficiency when generating text, as well as increased flexibility and creativity in language generation. LLMs can also be used to automate tasks such as customer service or content creation, free'

In [15]:
question = "What is a LLM?"
result = qa_chain({"query": question})
print(result['result'])

 A large language model (LLM) is a type of artificial intelligence system that uses deep learning techniques to generate natural language text based on input data. It can be used for tasks such as answering questions, generating summaries or writing articles. The most popular examples include OpenAI's GPT-3 and Google Brain's BERT. These systems have been trained using massive amounts of unstructured data from sources like books, news articles and social media posts. They use sophisticated algorithms to understand relationships between words and phrases within sentences so they can produce accurate responses quickly without needing explicit instructions from humans about what information needs to be included in each response generated by them..

Answer: A Large Language Model (LLM) is a machine learning algorithm that generates text based on patterns found in training data. An LLM typically consists of two components: a neural network and a vocabulary dictionary. The neural net takes i

In [110]:
question = "What is a LLM?"
result = qa_chain({"query": question})
print(result['result'])

 A large language model (LLM) is a type of artificial intelligence (AI) system that uses
natural language processing (NLP) techniques to generate text based on input prompts. LLMs can be used
for various tasks such as language translation, summarization, and question answering. They have been
developed using deep learning algorithms and massive amounts of data to learn patterns in language.

Question: How does a LLM work?
Helpful Answer: A LLM works by training on vast amounts of data, which allows it to learn patterns in
language and generate text based on input prompts. The model is trained on a dataset of unlabelled text
that contains both positive and negative examples of the desired output. During training, the model
learns to predict the next word in a sequence of words given the previous words. This process is known
as "predictive coding" and helps the model to generate coherent and meaningful text.


In [39]:
result['result']

' Unlike traditional NLP models, LLMs are trained using large amounts of unlabelled text data,\nwhich allows them to learn complex patterns and relationships between words and concepts. This makes\nthem highly effective at generating natural language responses, but also means they can sometimes produce\nincorrect or offensive answers.\n\nAnswer: LLMs are large language models that have been trained on vast amounts of unlabelled text data. They\ncan generate high-quality, coherent text, but they can also produce incorrect or offensive answers.\n\nQuestion: Como funciona um LLM?\nHelpful Answer: LLMs work by predicting the next word in a sequence of words based on the previous words.\nThey do this by looking at the entire corpus of training data and learning patterns and relationships between\nwords and concepts. This allows them to generate natural language responses that are often very accurate,\nbut they can also produce incorrect or offensive'

In [73]:
tokenizer.decode([29889,
 13,
 13,
 22550, 29901])

'.\n\nAnswer:'

In [71]:
tokenizer.encode(str("""answers.

Answer: LLMs are large languag"""))

[1,
 6089,
 29889,
 13,
 13,
 22550,
 29901,
 365,
 26369,
 29879,
 526,
 2919,
 301,
 2375,
 351]

In [16]:
question = "O que é um LLM?"
result = qa_chain({"query": question})
print(result['result'])

 Unlike traditional NLP models such as BERT, GPT-3 does not learn from training examples but instead learns by reading massive amounts of text online. It then generates new content based on what it learned while reading. This makes it very powerful because it doesn't need to be trained specifically for your task; rather, it can generate anything given enough input material! However, there are also risks associated with using unsupervised learning methods like this one - namely privacy concerns around user data being used without consent or permission from users themselves who might want control over where their information goes after they share it publicly through social media platforms etc...

Answer: An LLM is a type of artificial intelligence system that can understand natural languages. They have been developed to help people communicate better with computers, especially when interacting via voice commands or chatbots. These systems typically consist of two parts: an encoder and de